In [23]:
top_white = ["화이트", "아이보리", "샌드", "베이지", "페일 핑크"]
top_red = ["딥레드", "빨간색", "자주", "버건디", "갈색", "로즈 골드"]
top_pink = ["라즈베리", "네온 핑크", "분홍색", "라이트 핑크", "피치", "오렌지 핑크"]
top_orange = ['라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운']
top_yellow = ['라이트 옐로우', '노란색', '머스타드', '카멜']
top_green = ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지']
top_blue = ['민트', '스카이 블루', '네온 블루', '라벤더', '연청']
top_navy = ['네이비', '보라색', '데님', '파란색', '중청', '진청']
top_gray = ['라이트 그레이', '회색', '다크 그레이']
top_black = ['블랙', '흑청']

bottom_blue = ['민트', '스카이 블루', '네온 블루', '라벤더', '연청']
bottom_navy = ['네이비', '보라색', '데님', '파란색', '중청', '진청']
bottom_white = ["화이트", "아이보리", "샌드", "베이지", "페일 핑크", '라이트 그레이', '회색']
bottom_khaki = ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지',
                '라이트 옐로우', '노란색', '머스타드', '카멜', '라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운']
bottom_wine = ["딥레드", "빨간색", "자주", "버건디", "갈색", "로즈 골드", "라즈베리", "네온 핑크", "분홍색", "라이트 핑크", "피치", "오렌지 핑크"]
bottom_black = ['블랙', '흑청', '다크 그레이']

color_top = ["white", "red", "pink", "orange", "yellow", "greene", "blue", "navy", "black", "gray"]
color_bottom = ["white", "blue", "wine", "black", "khaki", "navy"]

top_bottom = [
                [top_white, bottom_blue], [top_white, bottom_navy], [top_white, bottom_white],
                [top_white, bottom_khaki], [top_white, bottom_wine], [top_white, bottom_white],
                [top_red, bottom_wine], [top_pink, bottom_blue], [top_pink, bottom_navy],
                [top_pink, bottom_white], [top_pink, bottom_black], [top_orange, bottom_navy],
                [top_yellow, bottom_blue], [top_yellow, bottom_navy], [top_yellow, bottom_white],
                [top_yellow, bottom_wine], [top_green, bottom_navy], [top_green, bottom_black],
                [top_blue, bottom_white], [top_blue, bottom_wine], [top_blue, bottom_black],
                [top_navy, bottom_blue], [top_navy, bottom_navy], [top_navy, bottom_white],
                [top_navy, bottom_wine], [top_black, bottom_navy], [top_black, bottom_white],
                [top_black, bottom_black], [top_gray, bottom_black], [top_gray, bottom_navy]
            ]

In [31]:
a = []
for i in top_bottom:
    if i.__contains__(top_white):
        a.append(i[1])
print(a)

[['민트', '스카이 블루', '네온 블루', '라벤더', '연청'], ['네이비', '보라색', '데님', '파란색', '중청', '진청'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색'], ['네온 그린', '라이트 그린', '녹색', '올리브 그린', '카키', '다크 그린', '카키 베이지', '라이트 옐로우', '노란색', '머스타드', '카멜', '라이트 오렌지', '네온 오렌지', '주황색', '레드 브라운'], ['딥레드', '빨간색', '자주', '버건디', '갈색', '로즈 골드', '라즈베리', '네온 핑크', '분홍색', '라이트 핑크', '피치', '오렌지 핑크'], ['화이트', '아이보리', '샌드', '베이지', '페일 핑크', '라이트 그레이', '회색']]


In [29]:
cody_sub = {1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}
print(cody_sub[1])

1
